Among all injection types, SQL injection is one of the most common attack vectors, and arguably the most dangerous. As Python is one of the most popular programming languages in the world, knowing how to protect against Python SQL injection is critical.

In this tutorial, you’re going to learn:

    What Python SQL injection is and how to prevent it
    How to compose queries with both literals and identifiers as parameters
    How to safely execute queries in a database

This tutorial is suited for users of all database engines. The examples here use PostgreSQL, but the results can be reproduced in other database management systems (such as SQLite, MySQL, Microsoft SQL Server, Oracle, and so on).

Generating and executing SQL queries is a common task. However, companies around the world often make horrible mistakes when it comes to composing SQL statements. While the ORM layer usually composes SQL queries, sometimes you have to write your own.

When you use Python to execute these queries directly into a database, there’s a chance you could make mistakes that might compromise your system. In this tutorial, you’ll learn how to successfully implement functions that compose dynamic SQL queries without putting your system at risk for Python SQL injection.

Setting Up a Database

To get started, you’re going to set up a fresh PostgreSQL database and populate it with data. Throughout the tutorial, you’ll use this database to witness firsthand how Python SQL injection works.
Creating a Database

First, open your shell and create a new PostgreSQL database owned by the user postgres:

In [ ]:
createdb -O postgres psycopgtest

Here you used the command line option -O to set the owner of the database to the user postgres. You also specified the name of the database, which is psycopgtest.

Note: postgres is a special user, which you would normally reserve for administrative tasks, but for this tutorial, it’s fine to use postgres. In a real system, however, you should create a separate user to be the owner of the database.

Your new database is ready to go! You can connect to it using psql:

psql -U postgres -d psycopgtest

In [1]:
import sqlite3

In [4]:
conn = sqlite3.connect('example.db')
c = conn.cursor()

In [3]:

c.execute('''CREATE TABLE users
             (username, admin)''')

In [4]:
c.execute('''INSERT INTO users VALUES ('Ashish',True),('Akash',False)''')
conn.commit()

In [5]:
result= c.execute('''SELECT COUNT(*) FROM users''').fetchone()

In [6]:
result

(2,)

In [32]:
# BAD EXAMPLE. DON'T DO THIS!
def is_admin(username):
    conn = sqlite3.connect('example.db')
    c = conn.cursor()
    c.executescript('''SELECT admin FROM users WHERE username = '%s' '''%username)
    result = c.fetchone()
    if result is None:
        return 0
    admin, = result
    return admin

In [26]:
is_admin('Ashish')

0

Exploiting Query Parameters With Python SQL Injection

In the previous example, you used string interpolation to generate a query. Then, you executed the query and sent the resulting string directly to the database. However, there’s something you may have overlooked during this process.

Think back to the username argument you passed to is_admin(). What exactly does this variable represent? You might assume that username is just a string that represents an actual user’s name. As you’re about to see, though, an intruder can easily exploit this kind of oversight and cause major harm by performing Python SQL injection.

Try to check if the following user is an admin or not:

In [33]:
is_admin("'; update users set admin = 0 where username = 'Akash'; select true; --")

0

So, why is this happening? Well, what do you know about the username argument? You know it should be a string representing the username, but you don’t actually check or enforce this assertion. This can be dangerous! It’s exactly what attackers are looking for when they try to hack your system.


In [17]:
print("select admin from users where username = '%s'" % "'; select true; --")

select admin from users where username = ''; select true; --'


In [18]:
result=c.execute('SELECT True')
result.fetchone()

(1,)

In [21]:
result=c.execute('SELECT username FROM users --WHERE admin=1')
result.fetchall()

[('Ashish',), ('Akash',)]

Crafting Safe Query Parameters

In the previous section, you saw how an intruder can exploit your system and gain admin permissions by using a carefully crafted string. The issue was that you allowed the value passed from the client to be executed directly to the database, without performing any sort of check or validation. SQL injections rely on this type of vulnerability.

Any time user input is used in a database query, there’s a possible vulnerability for SQL injection. The key to preventing Python SQL injection is to make sure the value is being used as the developer intended. In the previous example, you intended for username to be used as a string. In reality, it was used as a raw SQL statement.

To make sure values are used as they’re intended, you need to escape the value. For example, to prevent intruders from injecting raw SQL in the place of a string argument, you can escape quotation marks:

In [ ]:
# BAD EXAMPLE. DON'T DO THIS!
username = username.replace("'", "''")

This is just one example. There are a lot of special characters and scenarios to think about when trying to prevent Python SQL injection. Lucky for you, modern database adapters, come with built-in tools for preventing Python SQL injection by using query parameters. These are used instead of plain string interpolation to compose a query with parameters.

Note: Different adapters, databases, and programming languages refer to query parameters by different names. Common names include bind variables, replacement variables, and substitution variables.

Now that you have a better understanding of the vulnerability, you’re ready to rewrite the function using query parameters instead of string interpolation:

In [54]:
# BAD EXAMPLE. DON'T DO THIS!
def is_admin_2(username):
    conn = sqlite3.connect('example.db')
    c = conn.cursor()
    try:
        c.executescript('''SELECT admin FROM users WHERE username = %(username)s''', {'username': username})
    except:
        return 0
    result = c.fetchone()
    if result is None:
        return 0
    admin, = result
    return admin

In [55]:
is_admin_2("'; update users set admin = 2555 where username = 'Akash'; select true; --")

0

Here’s what’s different in this example:

In line 6, you used a named parameter username to indicate where the username should go. Notice how the parameter username is no longer surrounded by single quotation marks.

In line 6, you passed the value of username as the second argument to cursor.execute(). The connection will use the type and value of username when executing the query in the database.


The connection treated the value of username as a string and escaped any characters that might terminate the string and introduce Python SQL injection.

Using SQL Composition

So far you’ve used parameters for literals. Literals are values such as numbers, strings, and dates. But what if you have a use case that requires composing a different query—one where the parameter is something else, like a table or column name?

Inspired by the previous example, let’s implement a function that accepts the name of a table and returns the number of rows in that table:

In [71]:
# BAD EXAMPLE. DON'T DO THIS!
def count_rows(table_name):
    conn = sqlite3.connect('example.db')
    c = conn.cursor()
    c.execute("""SELECT count(*) FROM %(table_name)s""", {'table_name': table_name})
    result = c.fetchone()
    rowcount, = result
    return rowcount


In [72]:
count_rows('users')

OperationalError: near "%": syntax error

The command failed to generate the SQL. As you’ve seen already, the database adapter treats the variable as a string or a literal. A table name, however, is not a plain string. This is where SQL composition comes in.

You already know it’s not safe to use string interpolation to compose SQL. Luckily, Psycopg provides a module called psycopg.sql to help you safely compose SQL queries. Let’s rewrite the function using psycopg.sql.SQL():